In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [64]:
with open('./eagle_bestguess/snp50c_hd_merged.chr1.eagle.haps', 'r') as f:
    ehapsline = f.readline()
with open('./eagle_bestguess/snp50c_hd_merged.chr1.eagle.sample', 'r') as f:
    esamp = f.readlines()
with open('./impute2/out/impute2_imputed.gen', 'r') as f:
    igenline = f.readline()
with open('./impute2/out/impute2_imputed.sample', 'r') as f:
    isamp = f.readlines()

In [67]:
hapra=ehapsline.split(" ")[3:5]
genra=igenline.split(" ")[3:5]

In [69]:
hap=[float(i) for i in (ehapsline.split(" ")[5:])]
gen=[float(i) for i in (igenline.split(" ")[5:])]

In [70]:
genchunklist=zip(*[iter(gen)]*3)
genhardcalls=[]
for entry in genchunklist:
    genhardcalls.append([round(xx) for xx in entry])
hapchunklist=[list(x) for x in zip(*[iter(hap)]*2)]

#Convert to ref/alt alleles based on coding given by impute
for xx in range(0,len(genhardcalls)):
    if genhardcalls[xx] == [1.0, 0.0, 0.0]:
        genhardcalls[xx] = genra[0],genra[0]
    elif genhardcalls[xx] == [0.0, 1.0, 0.0]:
        genhardcalls[xx] = genra[0],genra[1]
    elif genhardcalls[xx] == [0.0, 0.0, 1.0]:
        genhardcalls[xx] = genra[1],genra[1]
    else:
        genhardcalls[xx]= "missing"

In [71]:
for xx in range(0,len(hapchunklist)):
    if hapchunklist[xx] == [0.0, 0.0]:
        hapchunklist[xx] = hapra[0],hapra[0]
    elif hapchunklist[xx] == [0.0, 1.0]:
        hapchunklist[xx] = hapra[0],hapra[1]
    elif hapchunklist[xx] == [1.0, 0.0]:
        hapchunklist[xx] = hapra[0],hapra[1]
    elif hapchunklist[xx] == [1.0, 1.0]:
        hapchunklist[xx] = hapra[1],hapra[1]
    else:
        hapchunklist[xx] = "missing"

In [72]:
x=Counter(str(entry) for entry in hapchunklist)
x

Counter({"('A', 'A')": 1413, "('A', 'B')": 1430, "('B', 'B')": 907})

In [73]:
x=Counter(str(entry) for entry in genhardcalls)
x

Counter({"('A', 'A')": 133,
         "('A', 'B')": 942,
         "('B', 'B')": 2204,
         'missing': 1})

In [401]:
#Create lists of ids in order from Sample list
genids=[]
for item in isamp[2:]:
    genids.append(item.split(" ")[1])
hapids=[]
for item in esamp[2:]:
    hapids.append(item.split(" ")[1])

In [402]:
#Dictionaries created for both methods.  Keys = ids, Values = method's respective "genotype"
indgen = {}
for x in range(0,len(genids)):
    indgen[genids[x]]=genhardcalls[x]
indhap = {}
for x in range(0,len(hapids)):
    indhap[hapids[x]]=hapchunklist[x]    

In [403]:
counter = 0
for xx in indhap.items():
    if xx in indgen.items():
        counter +=1
counter/float(len(indgen))

0.0009146341463414635